In [1]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get her0 if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

Current working directory: /Users/lucada/Documents/qhack/qhack21


In [2]:
from utils.data_handling import load_data, get_combined_cov_pos

In [3]:
data = load_data()[:100]
covs = get_combined_cov_pos(data)

Reading data... ok


In [4]:
from utils.linalg import so3_to_su2, to_so3
from encoding import encode 
import numpy as np
import pennylane as qml

In [5]:
wires = np.arange(5)
dev = qml.device('default.qubit', wires=wires)

@qml.qnode(dev)
def circ():
    encode(covs[0], wires=wires)
    return [qml.expval(qml.PauliX(wires=i)) for i in wires]

circ()
print(circ.draw())

 0: ──RX(8.15)─────────────────────────────────────┤ ⟨X⟩ 
 1: ──RX(10.1)─────────────────────────────────────┤ ⟨X⟩ 
 2: ──RX(21.8)─────────────────────────────────────┤ ⟨X⟩ 
 3: ──H─────────╭C──RZ(1.86)──RX(1.16)──RZ(-3.14)──┤ ⟨X⟩ 
 4: ────────────╰X─────────────────────────────────┤ ⟨X⟩ 

